# Pesky Persistent DB

In [1]:
!minikube start

😄  minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🏃  Updating the running docker "minikube" container ...
🐳  Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: default-storageclass, storage-provisioner
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


## Explore Example

In [3]:
%%bash
kubectl apply -f examples/nginx-deployment.yml
kubectl apply -f examples/nginx-service.yml

deployment.apps/nginx-deployment created


In [7]:
!kubectl get pods -o wide

NAME                                READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
nginx-deployment-57d84f57dc-cbrq8   1/1     Running   0          22s   10.244.0.4   minikube   <none>           <none>
nginx-deployment-57d84f57dc-gmkbv   1/1     Running   0          22s   10.244.0.5   minikube   <none>           <none>
nginx-deployment-57d84f57dc-rw652   1/1     Running   0          22s   10.244.0.6   minikube   <none>           <none>


In [9]:
!minikube service nginx-nodeport-service url

|-----------|------------------------|-------------|---------------------------|
| NAMESPACE |          NAME          | TARGET PORT |            URL            |
|-----------|------------------------|-------------|---------------------------|
| default   | nginx-nodeport-service |          80 | http://192.168.49.2:30000 |
|-----------|------------------------|-------------|---------------------------|
🎉  Opening service default/nginx-nodeport-service in default browser...
👉  http://192.168.49.2:30000


In [10]:
!curl -v http://192.168.49.2:30000

*   Trying 192.168.49.2:30000...
* TCP_NODELAY set
* Connected to 192.168.49.2 (192.168.49.2) port 30000 (#0)
> GET / HTTP/1.1
> Host: 192.168.49.2:30000
> User-Agent: curl/7.68.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.25.2
< Date: Mon, 09 Oct 2023 00:08:54 GMT
< Content-Type: text/html
< Content-Length: 615
< Last-Modified: Tue, 15 Aug 2023 17:03:04 GMT
< Connection: keep-alive
< ETag: "64dbafc8-267"
< Accept-Ranges: bytes
< 
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a hre

In [11]:
!minikube stop
!minikube delete

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 node stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /home/codespace/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


## Postgres

In [106]:
!minikube start

😄  minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
✨  Automatically selected the docker driver. Other choices: none, ssh
📌  Using Docker driver with root privileges
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔗  Configuring bridge CNI (Container Networking Interface) ...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🔎  Verifying Kubernetes components...
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


### Step 1

__Create nodeport service for `postgres`__

```yaml
apiVersion: v1
kind: Service
metadata:
  name: postgres
  labels:
    app: postgres
spec:
  type: NodePort
  ports:
    - port: 5432
  selector:
    app: postgres

```

In [107]:
!kubectl apply -f postgres-service.yml

service/postgres created


### Step 2

__Add database connection info to configmap__

In [108]:
%%writefile postgres-configmap.yml
apiVersion: v1
kind: ConfigMap
metadata:
  name: postgres-config
  labels:
    app: postgres
data:
  POSTGRES_DB: postgresdb
  POSTGRES_USER: admin
  POSTGRES_PASSWORD: psltest

Overwriting postgres-configmap.yml


In [109]:
!kubectl apply -f postgres-configmap.yml

configmap/postgres-config created


### Step 3

__Define PersistentVolumn (PV) and PersistentVolumeClaim (PVC)__

In [110]:
%%writefile postgres-persistent.yml
kind: PersistentVolume
apiVersion: v1
metadata:
  name: postgres-pv-volume
  labels:
    type: local
    app: postgres
spec:
  storageClassName: manual
  capacity: 
    storage: 5Gi
  accessModes: 
    - ReadWriteMany
  hostPath: 
    path: /mnt/data
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: postgres-pv-claim
  labels:
    app: postgres
spec:
  storageClassName: manual
  accessModes: 
    - ReadWriteMany
  resources: 
    requests:
      storage: 5Gi

Overwriting postgres-persistent.yml


In [111]:
!kubectl apply -f postgres-persistent.yml

persistentvolume/postgres-pv-volume created
persistentvolumeclaim/postgres-pv-claim created


__Tip__: Use these commands to get an explanation of the spec.

```bash
kubectl explain pv.spec --recursive
kubectl explain pvc.spec --recursive
```

### Step 4

__Configure the deloyment__

In [114]:
%%writefile postgres-deployment.yml
# TODO:
# a. Add a config map reference to your secrets
# b. Add the following volumeMounts:
#        mountPath: /var/lib/postgresql/data
#        name to be postgredb
# c. Add the persistent volume claim from postgres-persistent.yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: postgres
spec:
  replicas: 1
  selector:
    matchLabels:
      app: postgres
  template:
    metadata:
      labels:
        app: postgres
    spec:
      containers:
        - name: postgres
          image: postgres:10.1
          imagePullPolicy: "IfNotPresent"
          ports:
            - containerPort: 5432
          envFrom:
            - configMapRef:
                name: postgres-config
          volumeMounts:
            - name: postgresdb
              mountPath: /var/lib/postgresql/data
      volumes:
        - name: postgresdb
          persistentVolumeClaim:
            claimName: postgres-pv-claim


Overwriting postgres-deployment.yml


In [115]:
!kubectl apply -f postgres-deployment.yml

deployment.apps/postgres created


In [121]:
!kubectl get pod

NAME                        READY   STATUS    RESTARTS   AGE
postgres-6d8577568d-vggn9   1/1     Running   0          25s


In [62]:
!kubectl delete pod postgres-85989586bd-4wp4q

pod "postgres-85989586bd-4wp4q" deleted


In [41]:
!kubectl exec -it  postgres-6d8577568d-vggn9 -- psql -h localhost -U admin --password -p 5432 postgresdb

I1009 00:53:47.441068   83805 log.go:194] (0xc0000e64d0) (0xc000529860) Create stream
I1009 00:53:47.441139   83805 log.go:194] (0xc0000e64d0) (0xc000529860) Stream added, broadcasting: 1
I1009 00:53:47.444365   83805 log.go:194] (0xc0000e64d0) Reply frame received for 1
I1009 00:53:47.444400   83805 log.go:194] (0xc0000e64d0) (0xc000528000) Create stream
I1009 00:53:47.444417   83805 log.go:194] (0xc0000e64d0) (0xc000528000) Stream added, broadcasting: 3
I1009 00:53:47.445004   83805 log.go:194] (0xc0000e64d0) Reply frame received for 3
I1009 00:53:47.445044   83805 log.go:194] (0xc0000e64d0) (0xc0004ec000) Create stream
I1009 00:53:47.445055   83805 log.go:194] (0xc0000e64d0) (0xc0004ec000) Stream added, broadcasting: 5
I1009 00:53:47.445743   83805 log.go:194] (0xc0000e64d0) Reply frame received for 5
I1009 00:53:47.445770   83805 log.go:194] (0xc0000e64d0) (0xc0004ec0a0) Create stream
I1009 00:53:47.445783   83805 log.go:194] (0xc0000e64d0) (0xc0004ec0a0) Stream added, broadcasting

In [122]:
!kubectl describe pod postgres-6d8577568d-vggn9

Name:             postgres-6d8577568d-vggn9
Namespace:        default
Priority:         0
Service Account:  default
Node:             minikube/192.168.49.2
Start Time:       Mon, 09 Oct 2023 01:35:28 +0000
Labels:           app=postgres
                  pod-template-hash=6d8577568d
Annotations:      <none>
Status:           Running
IP:               10.244.0.3
IPs:
  IP:           10.244.0.3
Controlled By:  ReplicaSet/postgres-6d8577568d
Containers:
  postgres:
    Container ID:   docker://ade25e1aa53de299feddead61a3a3e37bde598498a82ee26a671a04b19891fb7
    Image:          postgres:10.1
    Image ID:       docker-pullable://postgres@sha256:3f4441460029e12905a5d447a3549ae2ac13323d045391b0cb0cf8b48ea17463
    Port:           5432/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Mon, 09 Oct 2023 01:35:41 +0000
    Ready:          True
    Restart Count:  0
    Environment Variables from:
      postgres-config  ConfigMap  Optional: false
    Environment:      

In [123]:
!kubectl describe deployment postgres

Name:                   postgres
Namespace:              default
CreationTimestamp:      Mon, 09 Oct 2023 01:35:28 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=postgres
Replicas:               1 desired | 1 updated | 1 total | 1 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=postgres
  Containers:
   postgres:
    Image:      postgres:10.1
    Port:       5432/TCP
    Host Port:  0/TCP
    Environment Variables from:
      postgres-config  ConfigMap  Optional: false
    Environment:       <none>
    Mounts:
      /var/lib/postgresql/data from postgresdb (rw)
  Volumes:
   postgresdb:
    Type:       PersistentVolumeClaim (a reference to a PersistentVolumeClaim in the same namespace)
    ClaimName:  postgres-pv-claim
    ReadOnly:   false
Conditions:
  Type           Status  Rea

### Challenge

In [2]:
!echo -n postgresdb | base64
!echo  -n admin | base64
!echo -n psltest | base64


cG9zdGdyZXNkYg==
YWRtaW4=
cHNsdGVzdA==


In [14]:
%%writefile postgres-secret.yml
apiVersion: v1
kind: Secret
metadata:
  name: postgres-secret
type: Opaque
data:
  POSTGRES_USER: YWRtaW4=
  POSTGRES_PASSWORD: cHNsdGVzdA==

Overwriting postgres-secret.yml


In [15]:
%%writefile postgres-deployment.yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: postgres
spec:
  replicas: 1
  selector:
    matchLabels:
      app: postgres
  template:
    metadata:
      labels:
        app: postgres
    spec:
      containers:
        - name: postgres
          image: postgres:10.1
          imagePullPolicy: "IfNotPresent"
          ports:
            - containerPort: 5432
          envFrom:
            - configMapRef:
                name: postgres-config
          # Reference secrets from postgres-secret.yml
          env:
            - name: POSTGRES_USER
              valueFrom:
                secretKeyRef:
                  name: postgres-secret
                  key: POSTGRES_USER
            - name:  POSTGRES_PASSWORD
              valueFrom:
                secretKeyRef:
                  name: postgres-secret
                  key: POSTGRES_PASSWORD
          volumeMounts:
            - name: postgresdb
              mountPath: /var/lib/postgresql/data
              # Mount the secrets as a volume
            - name: postgres-secret-volume
              mountPath: /etc/secrets
              readOnly: true
      volumes:
        - name: postgresdb
          persistentVolumeClaim:
            claimName: postgres-pv-claim
        - name: postgres-secret-volume
          secret:
            secretName: postgres-secret

Overwriting postgres-deployment.yml


In [16]:
%%bash
# minikube start
kubectl apply -f postgres-service.yml
kubectl apply -f postgres-persistent.yml
kubectl apply -f postgres-configmap.yml
kubectl apply -f postgres-secret.yml
kubectl apply -f postgres-deployment.yml

service/postgres unchanged
persistentvolume/postgres-pv-volume unchanged
persistentvolumeclaim/postgres-pv-claim unchanged
configmap/postgres-config configured
secret/postgres-secret configured
deployment.apps/postgres configured


In [85]:
import subprocess
import yaml
def run_checks(gh_actions_file: str):
    with open(gh_actions_file) as f:
        actions = yaml.safe_load(f)
    steps = actions['jobs']['scavenger']['steps']
    checks = [step for step in steps if 'Check ' in step['name']]
    for check in checks:
        subprocess.run(check["run"], capture_output=False, shell=True, executable='/bin/bash')

run_checks('/workspaces/intro-to-kube/.github/workflows/pesky-persistent-db.yml')

✅ Value for POSTGRES_DB is correct
✅ Value for POSTGRES_PASSWORD is correct
✅ Value for POSTGRES_USER is correct
✅ Access Mode for Persistent Volume is correct
✅ Capacity for Persistent Volume is correct
✅ StorageClass for Persistent Volume is correct
✅ Host Path for Persistent Volume is correct
RWX
✅ Access Mode for Persistent Volume Claim is correct
✅ Capacity for Persistent Volume Claim is correct
✅ StorageClass for Persistent Volume Claim is correct
✅ Deployment ConfigMap setup is correct
✅ Deployment Persistent Volume Claim setup is correct
